1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.


In [233]:
import selenium 
import pandas as pd
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import os
import urllib
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')

In [234]:
drivers= webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")

In [235]:
drivers.get("https://www.amazon.in/")

In [236]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
productname= input("Enter the product name : ")
search_item.send_keys(productname)

#Click Search Button
search_btn=drivers.find_element_by_xpath('//input[@id="nav-search-submit-button"]')
search_btn.click()

Enter the product name : guitar


2.In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [237]:
Brand_Name = []
Name_of_the_Product = []
Rating = []
No_of_Ratings = []
Price = []
Return_Exchange = []
Expected_Delivery = []
Availability = []
Other_Details = []
URL = []

In [238]:
j=1
while j<=3:    
    product_url=drivers.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
    for i in product_url:
        URL.append(i.get_attribute('href'))    
    try:
        drivers.find_elements_by_xpath('//div[@class="a-text-center"]/ul/li')[-1].click() 
        time.sleep(2)
    except:
        print("End")    
    j+=1

In [239]:
for url in URL:
    drivers.get(url)
#Brand_Name        
    try:
        brand=drivers.find_element_by_xpath('//table[@id="productDetails_techSpec_section_1"]/tb')[1]
        Brand_Name.append(brand.text)
        
    except NoSuchElementException as e:
        Brand_Name.append("-")
#Return_Exchange                                            
    try:
        Return=drivers.find_element_by_xpath('//div[@id="RETURNS_POLICY"]')
        Return_Exchange.append(Return.text)
        
    except NoSuchElementException as e:
        Return_Exchange.append("-")
#Expected_Delivery                                            
    try:
        delivery=drivers.find_element_by_xpath('//span[@id="upsell-message"]/b')
        Expected_Delivery.append(delivery.text)  
        
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
#Availability
    try:
        availb=drivers.find_element_by_xpath('//span[@class="a-size-medium a-color-success"]')
        Availability.append(availb.text)  
        
    except NoSuchElementException as e:
        Availability.append("-")
#Other_Details
    try:
        otherd=drivers.find_element_by_xpath('//div[@id="productDescription"]')
        Other_Details.append(otherd.text)
        
    except NoSuchElementException as e:
        Other_Details.append("-")
#price
    try:
        price=drivers.find_element_by_xpath('//span[@id="priceblock_ourprice"]')
        Price.append(price.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Price.append("-")
#Name of the product
    try:
        Name_tags=drivers.find_element_by_xpath('//span[@id="productTitle"]')
        Name_of_the_Product.append(Name_tags.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Name_of_the_Product.append("-")
        
#Rating
#No_of_Ratings                                            
    try:
        rate=drivers.find_element_by_xpath('//span[@id="acrCustomerReviewText"]')
        No_of_Ratings.append(rate.text)        # No of Ratings                            
        WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//span[@id="acrCustomerReviewText"]'))).click()
        rating=drivers.find_element_by_xpath('//span[@class="a-size-medium a-color-base"]')
        Rating.append(rating.text)     # Collecting Ratings
        
    except NoSuchElementException as e:
        Rating.append("-") 
        No_of_Ratings.append("-")

In [240]:
data= list(zip(Name_of_the_Product,Rating,No_of_Ratings,Price,Return_Exchange,Expected_Delivery,Availability,Other_Details,URL,Brand_Name))

In [241]:
df= pd.DataFrame(data,columns=["Name_of_the_Product","Rating","No_of_Ratings","Price","Return_Exchange","Expected_Delivery","Availability","Other_Details","Product_URL","Brand_Name"])

In [242]:
df.head(50)

,Name_of_the_Product,Rating,No_of_Ratings,Price,Return_Exchange,Expected_Delivery,Availability,Other_Details,Product_URL,Brand_Name
0,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4.1 out of 5,310 ratings,-,7 Days Replacement,"Monday, Dec 27",In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...,-
1,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",3.9 out of 5,"1,311 ratings",-,7 Days Replacement,-,-,If you are a music lover and a fan of acoustic...,https://www.amazon.in/gp/slredirect/picassoRed...,-
2,Kadence Frontier guitar with Online Guitar lea...,4 out of 5,"1,247 ratings",-,7 Days Replacement,"Monday, Dec 27",In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...,-
3,Vault ST1 Premium Electric Guitar - White,3.9 out of 5,28 ratings,-,7 Days Replacement,-,In stock.,-,https://www.amazon.in/gp/slredirect/picassoRed...,-
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"14,404 ratings",-,7 Days Replacement,-,In stock.,-,https://www.amazon.in/Juarez-Acoustic-Cutaway-...,-
5,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",4 out of 5,"8,588 ratings",-,7 Days Replacement,-,-,-,https://www.amazon.in/Intern-INT-38C-Acoustic-...,-
6,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",3.9 out of 5,"1,311 ratings",-,7 Days Replacement,-,-,If you are a music lover and a fan of acoustic...,https://www.amazon.in/Juarez-Acoustic-Guitar-C...,-
7,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",4 out of 5,"8,588 ratings",-,7 Days Replacement,-,-,-,https://www.amazon.in/Intern-INT-38C-Acoustic-...,-
8,Kadence Frontier guitar with Online Guitar lea...,4 out of 5,"1,247 ratings",-,7 Days Replacement,"Monday, Dec 27",In stock.,-,https://www.amazon.in/Kadence-Frontier-Acousti...,-
9,"Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 3...",4 out of 5,"4,838 ratings",-,7 Days Replacement,-,In stock.,-,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...,-


3.Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [243]:
drivers= webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get("https://images.google.com/?gws_rd=ssl")

In [244]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
productname= input("Enter the keyword : ")
search_item.send_keys(productname)

#Click Search Button
search_btn=drivers.find_element_by_xpath('//span[@class="z1asCe MZy1Rb"]')
search_btn.click()

Enter the keyword : fruits


In [245]:
path = r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe"

url_prefix = "https://www.google.com.sg/search?q="
url_postfix = "&source=lnms&tbm=isch&sa=X&ei=0eZEVbj3IJG5uATalICQAQ&ved=0CAcQ_AUoAQ&biw=939&bih=591"

save_folder = 'Train Images'

def main():
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    download_images()
    
def download_images():
    topic = input("What do you want to search for? ")
    n_images = int(input('How many images do you want? '))
    
    search_url = url_prefix+topic+url_postfix
    #print(search_url)
    
    path = r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe"
    
    driver = webdriver.Chrome(path)
    driver.get(search_url)
    
    value = 0
    for i in range(8):
        driver.execute_script("scrollBy("+ str(value) +",+1000);")
        value += 1000
        time.sleep(2)
    
    elem1 = driver.find_element_by_id('islmp')
    sub = elem1.find_elements_by_tag_name('img')
    
    for j,i in enumerate(sub):
        if j <= n_images:
            src = i.get_attribute('src')                         
            try:
                if src != None:
                    src  = str(src)
                    
                    urllib.request.urlretrieve(src, os.path.join(save_folder, topic+str(j)+'.jpg'))
                    
                else:
                    raise TypeError
            except Exception as e:              #catches type error along with other errors
                print(f'fail with error {e}')
    
    driver.close()
    
if __name__ == "__main__":
    main()

What do you want to search for? fruits
How many images do you want? 10


4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [246]:
drivers= webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get("https://www.flipkart.com/")

In [247]:
search_btn=drivers.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')
search_btn.click()

In [248]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@class="_3704LK"]')
productname= input("Enter the mobile brand name : ")
search_item.send_keys(productname + " mobiles")

#Click Search Button
search_btn=drivers.find_element_by_xpath('//button[@class="L0Z3Pu"]')
search_btn.click()

Enter the mobile brand name : samsung


In [249]:
Brand_Name =[]
Smartphone_name = []
Colour = []
RAM = []
Storage = []
Primary_Camera = []
Secondary_Camera = []
Display_Size = []
Display_Resolution = []
Processor = []
Processor_Cores = []
Battery_Capacity = []
Price = []
Product_URL = []

In [250]:
#
product_url=drivers.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for i in product_url:
        Product_URL.append(i.get_attribute('href'))
#        
for url in Product_URL:
    drivers.get(url)
#Brand_Name        
    try:
        brand=drivers.find_element_by_xpath('//span[@class="B_NuCI"]')
        Brand_Name.append(brand.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Brand_Name.append("-")
#Clicking on Read More
        
    try:
        WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@class="_2KpZ6l _1FH0tX"]'))).click()
    except NoSuchElementException as e:
        print('Not Clickable')    
# Smartphone_name                                            
    try:
#         sname=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li')
        sname=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li')
        Smartphone_name.append(sname.text)
        
    except NoSuchElementException as e:
        Smartphone_name.append("-")
#Colour                                            
    try:
        colour=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li')
        Colour.append(colour.text)  
        
    except NoSuchElementException as e:
        Colour.append("-")
#RAM
    try:
        ram=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(ram.text)
        
    except NoSuchElementException as e:
        RAM.append("-")
#Storage
    try:
        rom=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li')
        Storage.append(rom.text)
        
    except NoSuchElementException as e:
        Storage.append("-")
#price
    try:
        price=drivers.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(price.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Price.append("-")
#Primary_Camera
    try:
        pcam=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pcam.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Primary_Camera.append("-")
#Secondary_Camera
    try:
        scam=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[5]/table/tbody/tr[5]/td[2]/ul/li')
        Secondary_Camera.append(scam.text.strip(" "))
        
    except NoSuchElementException as e:
        Secondary_Camera.append("-")
#Display_Size
    try:
        display=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li')
        Display_Size.append(display.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Display_Size.append("-")
#Display_Resolution
    try:
        displayres=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[2]/table/tbody/tr[2]/td[2]/ul/li')
        Display_Resolution.append(displayres.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Display_Resolution.append("-")
#Processor
    try:
        processor=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[3]/table/tbody/tr[2]/td[2]/ul/li')
        Processor.append(processor.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Processor.append("-")
#Processor_Cores
    try:
        cores=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[3]/table/tbody/tr[3]/td[2]/ul/li')
        Processor_Cores.append(cores.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Processor_Cores.append("-")
#Battery_Capacity
    try:
        battery=drivers.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[8]/div[4]/div/div[2]/div[1]/div[10]/table/tbody/tr/td[2]/ul/li')
        Battery_Capacity.append(battery.text.strip(" ").replace('\n',''))
        
    except NoSuchElementException as e:
        Battery_Capacity.append("-")                                             
                                        

In [251]:
data= list(zip(Brand_Name,Smartphone_name,Price,Colour,RAM,Storage,Primary_Camera,Secondary_Camera,Display_Size,Display_Resolution,Processor,Processor_Cores,Battery_Capacity,Product_URL))


In [253]:
df= pd.DataFrame(data,columns=["Brand_Name","Smartphone_name","Price","Colour","RAM","Storage","Primary_Camera","Secondary_Camera","Display_Size","Display_Resolution","Processor","Processor_Cores","Battery_Capacity","Product_URL"])
df[7:]

,Brand_Name,Smartphone_name,Price,Colour,RAM,Storage,Primary_Camera,Secondary_Camera,Display_Size,Display_Resolution,Processor,Processor_Cores,Battery_Capacity,Product_URL
7,"SAMSUNG Galaxy M12 (Black, 64 GB) (4 GB RAM)",Galaxy M12,"₹11,197",Black,4 GB,64 GB,Primary Camera,-,16.51 cm (6.5 inch),720*1600$$pixels,Octa Core,2 GHz,-,https://www.flipkart.com/samsung-galaxy-m12-bl...
8,"SAMSUNG Galaxy M31 (Space Black, 128 GB) (8 G...",Galaxy M31,"₹19,467",Space Black,8 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,9 mm,https://www.flipkart.com/samsung-galaxy-m31-sp...
9,"SAMSUNG M32 5G (Slate Black, 128 GB) (6 GB RAM)",M32 5G,"₹19,090",Slate Black,6 GB,128 GB,48MP + 8MP + 5MP + 2MP,13MP Front Camera,16.51 cm (6.5 inch),720 X 1600$$Pixels,Dimensity 720 5G,Octa Core,1 Year Warranty Provided by the Manufacturer f...,https://www.flipkart.com/samsung-m32-5g-slate-...
10,"SAMSUNG Galaxy A22 5G (Mint, 128 GB) (6 GB RAM)",Galaxy A22 5G,"₹18,390",Mint,6 GB,128 GB,48MP + 5MP + 2MP,8MP Front Camera,16.76 cm (6.6 inch),2408 x 1080 Pixels,MediaTek Dimensity 700 (MT6833V),Octa Core,5000 mAh,https://www.flipkart.com/samsung-galaxy-a22-5g...
11,SAMSUNG Guru 1200 (Indigo Blue),-,"₹1,310",-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/samsung-guru-1200/p/i...
12,"SAMSUNG Galaxy M32 5G (Sky Blue, 128 GB) (8 G...",Galaxy M32 5G,"₹21,679",Sky Blue,8 GB,128 GB,48MP + 8MP + 5MP + 2MP,13MP Front Camera,16.51 cm (6.5 inch),720 X 1600$$pixels,Dimensity 720 5G,Octa Core,1 Year Warranty Provided by the Manufacturer f...,https://www.flipkart.com/samsung-galaxy-m32-5g...
13,"SAMSUNG Galaxy F12 (Sky Blue, 128 GB) (4 GB RAM)",-,"₹12,499",-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/samsung-galaxy-f12-sk...
14,"SAMSUNG Galaxy M12 (Blue, 64 GB) (4 GB RAM)",Galaxy M12,"₹11,275",Blue,4 GB,64 GB,Primary Camera,-,16.51 cm (6.5 inch),720*1600$$pixels,Octa Core,2 GHz,-,https://www.flipkart.com/samsung-galaxy-m12-bl...
15,"SAMSUNG Galaxy F12 (Sea Green, 128 GB) (4 GB ...",-,"₹12,499",-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/samsung-galaxy-f12-se...
16,"SAMSUNG Galaxy A70s (Prism Crush White, 128 GB...",-,"₹19,999",-,-,-,-,-,-,-,-,-,-,https://www.flipkart.com/samsung-galaxy-a70s-p...


5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [257]:
drivers= webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get("https://www.google.co.in/")

In [258]:
#Input Search
search_item=drivers.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
city_name= input("Enter the City Name you want to know the geospatial coordinates: ")
search_item.send_keys("latitude and longitude coordinates of " + city_name )

#Click Search Button
search_btn=drivers.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]')
search_btn.click()

# WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//input[@class="gNO89b"][1]'))).send_keys(Keys.ENTER)

try:
    cord=drivers.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div[1]/div/div[1]/div[1]/div/div[2]/div/div/div/div[1]')
    print('Geospatial coordinates of '+ city_name +' '+ cord.text)
        
except NoSuchElementException as e:
    print('City not found')

Enter the City Name you want to know the geospatial coordinates: pune
Geospatial coordinates of pune 18.5204° N, 73.8567° E


6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [261]:
# site = 'https://trak.in/india-startup-funding-investment-2015/' 
site = 'https://trak.in/' 


In [266]:
drivers = webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get(site)
#Clicking Funding details options
try:
    WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//li[@class="bs-pretty-tabs-container menu-item-has-children menu-item better-anim-fade bsm-leave"]'))).click()
    WebDriverWait(drivers, 10).until(EC.element_to_be_clickable((By.XPATH, '//li[@id="menu-item-51510"]'))).click()
except:
    WebDriverWait(drivers, 20).until(EC.element_to_be_clickable((By.XPATH, '//li[@id="menu-item-51510"]'))).click()


#getting the page source 
time.sleep(2)
html = drivers.page_source
#Reading the tabels in HTML
df = pd.read_html(html)
#Storing the monthly data
jan = df[7]
feb = df[5]
mar = df[3]
#Concating the whole data 
df_row_reindex = pd.concat([jan, feb, mar], ignore_index=True)
df_row_reindex.drop("Sr. No.",axis='columns', inplace=True)
df_row_reindex


,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,18000000.0
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,6172258.5
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,30000000.0
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,15000000.0
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,16000000.0
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,15000000.0
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,100000000.0
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,4000000.0
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,2500000.0
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,250000000.0


7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [472]:
site = 'https://www.digit.in/'

In [502]:
drivers = webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get('https://www.digit.in/')
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="menu"]/ul/li[4]/a'))).click()
filter_btn=drivers.find_element_by_xpath('//button[@class="tablinks "][1]')
filter_btn.click()
cat_btn=drivers.find_element_by_xpath('//div[@id="laptops"]/div[3]')
cat_btn.click()
html = drivers.page_source
df = pd.read_html(html)

In [503]:
main= df[0]
main

,Product Name,Seller,Price
0,Acer NITRO 5 Ryzen 9 (2021),Flipkart,"₹ 129,990"
1,MSI Stealth 15M 11th Gen Core i7-11375H (2021,Flipkart,"₹ 134,990"
2,ASUS ROG Strix Scar 15 Ryzen 9-5900HX (2021),Flipkart,"₹ 268,990"
3,Alienware Area 51M R2,NaN,"₹ 342,989"
4,Alienware m15 R3,Amazon,"₹ 319,990"
5,ASUS ROG Strix Scar 15,Flipkart,"₹ 215,990"
6,Asus ROG Zephyrus G14,Amazon,"₹ 164,990"
7,Lenovo Legion 5i,Amazon,"₹ 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,Amazon,"₹ 185,000"
9,Acer Aspire 7 gaming,Amazon,"₹ 57,500"


In [506]:
# # specifications of Laptop from 1-10
lapt1 = df[1]
# lapt1.drop('SPECIFICATION',axis='columns',inplace= True)
lapt1 = lapt1.T
headers = lapt1.iloc[0]
lapt1  = pd.DataFrame(lapt1.values[1:], columns=headers)

lapt2 = df[3]
# lapt2.drop('SPECIFICATION',axis='columns',inplace= True)
lapt2 = lapt2.T
headers = lapt2.iloc[0]
lapt2  = pd.DataFrame(lapt2.values[1:], columns=headers)

lapt3 = df[5]
# lapt3.drop('SPECIFICATION',axis='columns',inplace= True)
lapt3 = lapt3.T
headers = lapt3.iloc[0]
lapt3  = pd.DataFrame(lapt3.values[1:], columns=headers)

lapt4 = df[7]
# lapt4.drop('SPECIFICATION',axis='columns',inplace= True)
lapt4 = lapt4.T
headers = lapt4.iloc[0]
lapt4  = pd.DataFrame(lapt4.values[1:], columns=headers)

lapt5 = df[8]
# lapt5.drop('SPECIFICATION.2',axis='columns',inplace= True)
lapt5 = lapt5.T
headers = lapt5.iloc[0]
lapt5  = pd.DataFrame(lapt5.values[1:], columns=headers)


In [516]:
df1[1::2]

,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
1,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,NaN
1,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,NaN
1,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,NaN
1,Windows 10 Home,"17.3"" (1920 x 1080)",10th Gen Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"₹342,989"
1,Windows 10 Home,"15.6"" (3840 x 2160)",10th Gen Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NaN,NaN,NaN,NaN


8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [284]:
site = 'https://www.forbes.com/'

In [285]:
drivers = webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get('https://www.forbes.com/')

In [286]:
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/header/nav/div[1]/button[1]'))).click()

In [287]:
WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/header/nav/div[3]/ul/li[1]/a'))).click()

In [288]:

WebDriverWait(drivers, 5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="row-1"]/div/div/div[1]/div/div[1]/div[1]/div[1]/a'))).click()

In [373]:
Rank = []
Name= []
Net_worth= []
Age= []
Citizenship=[]
Source=[]
Industry=[]


In [374]:
rank=drivers.find_elements_by_xpath('//*[@class="rank"]')
name=drivers.find_elements_by_xpath('//*[@class="personName"]')
net_worth=drivers.find_elements_by_xpath('//*[@class="netWorth"]')
age=drivers.find_elements_by_xpath('//*[@class="age"]')
citizenship=drivers.find_elements_by_xpath('//*[@class="countryOfCitizenship"]')
source=drivers.find_elements_by_xpath('//*[@class="source"]')
industry=drivers.find_elements_by_xpath('//*[@class="category"]')


In [375]:
for i in rank:
     Rank.append(i.text.replace("\n",' '))
for i in name:
     Name.append(i.text.replace("\n",''))
for i in net_worth:
     Net_worth.append(i.text.replace("\n",''))
for i in age:
     Age.append(i.text.replace("\n",''))
for i in citizenship:
     Citizenship.append(i.text.replace("\n",''))
for i in source:
     Source.append(i.text.replace("\n",''))
for i in industry:
     Industry.append(i.text.replace("\n",''))

In [377]:
Rank,Name
Net_worth
Age
Citizenship
Source
Industry
data= list(zip(Rank,Name,Net_worth,Age,Citizenship,Source,Industry))

In [385]:
df_forbes= pd.DataFrame(data,columns=["Rank","Name","Net_worth","Age","Citizenship","Source","Industry"])

In [386]:
df_forbes

,Rank,Name,Net_worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197.,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197.,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197.,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video.

In [518]:
import csv
import io
from selenium import webdriver
from selenium.common import exceptions
import sys
import time

In [529]:
def scrape(url):
    driver = webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")

    # Navigates to the URL, maximizes the current window, and
    # then suspends execution for (at least) 5 seconds (this
    # gives time for the page to load).
    driver.get('https://www.youtube.com/watch?v=u34gHaRiBIU')
    driver.maximize_window()
    time.sleep(5)
    try:
        # Extract the elements storing the video title and
        # comment section.
        title = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
        comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
    except exceptions.NoSuchElementException:
        # Note: Youtube may have changed their HTML layouts for
        # videos, so raise an error for sanity sake in case the
        # elements provided cannot be found anymore.
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll down 'til "next load".
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    # One last scroll just in case.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    try:
        # Extract the elements storing the usernames and comments.
        username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
        comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
    except exceptions.NoSuchElementException:
        error = "Error: Double check selector OR "
        error += "element may not yet be on the screen at the time of the find operation"
        print(error)
    print("> VIDEO TITLE: " + title + "\n")

    with io.open('results.csv', 'w', newline='', encoding="utf-16") as file:
        writer = csv.writer(file, delimiter =",", quoting=csv.QUOTE_ALL)
        writer.writerow(["Username", "Comment"])
        for username, comment in zip(username_elems, comment_elems):
            writer.writerow([username.text, comment.text])
    driver.close()
if __name__ == "__main__":
    scrape(sys.argv[1])

> VIDEO TITLE: THE BATMAN - The Bat and The Cat Trailer



10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [530]:
drivers = webdriver.Chrome(r"C:\Users\Sanjana\Downloads\chromedriver_win32\chromedriver.exe")
drivers.get('https://www.hostelworld.com/')

In [531]:
#Click Search Button
search_btn=drivers.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/div[5]/section/div/div[1]/a')
search_btn.click()

In [624]:
Hostel_name = []
Distance= []
Ratings= []
Reviews= []
Price=[]
Dorms=[]
Facilities=[]
Description=[]

In [628]:
distance=drivers.find_elements_by_xpath('//*[@class="resultheader rounded clearfix small-12 medium-7 large-8 columns clearfix"]/div[3]')
hostel_name=drivers.find_elements_by_xpath('//*[@data-name]/div[3]/h2')
ratings=drivers.find_elements_by_xpath('//*[@class="hwta-rating-summary"]/a')
reviews=drivers.find_elements_by_xpath('//*[@class="hwta-rating-info"]/price')
price=drivers.find_elements_by_xpath('//*[@class="fabresult-prices small-12"]')
facilities=drivers.find_elements_by_xpath('//*[@class="hwta-rating-factors clearfix"]')
description=drivers.find_elements_by_xpath('//*[@class="show-for-medium-up"]')


In [629]:
for i in distance:
     Distance.append(i.text.replace("\n",' '))
for i in hostel_name:
     Hostel_name.append(i.text.replace("\n",''))
for i in ratings:
     Ratings.append(i.text.replace("\n",''))
for i in reviews:
     Reviews.append(i.text.replace("\n",''))
for i in price:
    a=i.text.replace("Prices from:",'')
    Price.append(a.replace("\n",''))
for i in facilities:
     Facilities.append(i.text.replace("\n",''))
for i in description:
     Description.append(i.text.replace("\n",''))

In [630]:
Distance
Hostel_name
Reviews
Ratings
Price
Facilities
Description

["Wombat's The City Hostel London is definitely the place to stay if you want to experience all that London has to offer. Only minutes away from sights such as the Tower Bridge, the Tower of London, and... More...",
 "COVID 19 Policy Update.In response to Coronavirus (COVID-19), additional safety and sanitation measures are currently in effect at this property, including in-house disinfection.St Christopher'... More...",
 'With the British Museum literally on its doorstep, it is impossible to beat this hostel’s location! Located in the very heart of Central London you are a short walk from iconic attractions like... More...',
 'Safestay at Elephant & Castle is ideal if you are looking for budget accommodation in London, whether you see the sights in this amazing city or travelling with your family. If Tate Modern, the N... More...',
 "Generator London is a design hotel-hostel located in Russell Square in the heart of central London, a short stroll to Covent Garden, Regents Park and tre

In [635]:
data= list(zip(Hostel_name,Distance,Reviews,Ratings,Price,Facilities,Description))

In [636]:
df_hostel= pd.DataFrame(data,columns=["Hostel_name","Distance","Reviews","Ratings","Price","Facilities","Description"])

In [637]:
df_hostel

,Hostel_name,Distance,Reviews,Ratings,Price,Facilities,Description
0,Wombat's The City Hostel London,3.6km from city centre - SHOW ON MAP,Fabulous,8.8,INR1989.44,Wonderful LocationMarvellous StaffExcellent Cl...,Wombat's The City Hostel London is definitely ...
1,St Christopher's Village,1.8km from city centre - SHOW ON MAP,Fabulous,8.2,INR1296.15,Perfect LocationMarvellous Staff,COVID 19 Policy Update.In response to Coronavi...
2,Astor Museum Inn,2.2km from city centre - SHOW ON MAP,Superb,9.0,INR1607.63,Perfect LocationSuperb StaffExcellent Cleanliness,With the British Museum literally on its doors...
3,Safestay London Elephant & Castle,1.7km from city centre - SHOW ON MAP,Fabulous,8.0,INR1406.68,Wonderful Location,Safestay at Elephant & Castle is ideal if you ...
4,Generator London,3km from city centre - SHOW ON MAP,Very Good,7.8,INR1431.79,Wonderful LocationMarvellous Staff,Generator London is a design hotel-hostel loca...
5,No.8 Willesden Hostel London,10km from city centre - SHOW ON MAP,Very Good,7.5,INR507.41,Marvellous Staff,At No.8 we take pride in our Customer Service ...
6,Urbany Hostel London,5.4km from city centre - SHOW ON MAP,Superb,9.6,INR1909.06,Perfect LocationSuperb StaffFantastic Cleanliness,"Welcome to Urbany Hostel London, our first int..."
7,London Backpackers,11.9km from city centre - SHOW ON MAP,Fabulous,8.4,INR1105.24,Wonderful LocationMarvellous StaffExcellent Cl...,IMPORTANT NOTICE: This hostel is for 18 - 35-y...
8,The Walrus Hostel,0.2km from city centre - SHOW ON MAP,Fabulous,8.2,INR2310.97,Perfect LocationMarvellous Staff,The Walrus Hostel is the ideal choice for you ...
9,Smart Camden Inn Hostel,4.4km from city centre - SHOW ON MAP,Fabulous,8.4,INR1351.41,Perfect LocationSuperb Staff,"Located in the middle of Camden Town, we’re th..."
